## Misclassification cost as part of training

[Machine Learning with Imbalanced Data - Course](https://www.trainindata.com/p/machine-learning-with-imbalanced-data)


There are 2 ways in which we can introduce cost into the learning function of the algorithm with Scikit-learn:

- Defining the **class_weight** parameter for those estimators that allow it, when we set the estimator
- Passing a **sample_weight** vector with the weights for every single observation, when we fit the estimator.


With both the **class_weight** parameter or the **sample_weight** vector, we indicate that the loss function should be modified to accommodate the class imbalance and the cost attributed to each misclassification.

## parameters

**class_weight**: can take 'balanced' as argument, in which case it will use the balance ratio as weight. Alternatively, it can take a dictionary with {class: penalty}, pairs. In this case, it penalizes mistakes in samples of class[i] with penalty[i].

So if class_weight = {0:1, and 1:10}, misclassification of observations of class 1 are penalized 10 times more than misclassification of observations of class 0.

**sample_weight** is a vector of the same length as y, containing the weight or penalty for each individual observation. In principle, it is more flexible, because it allows us to set weights to the observations and not to the class as a whole. So in this case, for example we could set up higher penalties for fraudulent applications that are more costly (money-wise) than to those fraudulent applications that are of little money.

## Important

If you use both class_weight and sample_weight, the final penalty will be **the combination of the 2**, so be very careful

## Demo

In this demo, I will introduce cost-sensitive learning to Logistic Regression. But keep in mind that you can do the same with almost every other classifier in Scikit-learn using **sample_weight** or, using **Class_weight** in those estimators that have that attribute.

## Classifiers that support class_weight

In [1]:
# import sklearn
# sklearn.__version__

In [2]:
# NOTE, THIS CELL WILL NOT WORK WITH SKLEARN VERSION > 0.24
# JUST COMMENT IT OUT, OR DELETE THIS CELL TO CARRY ON WITH THE NOTEBOOK.

# Let's find out which classifiers from sklearn support class_weight
# as part of the __init__ method, that is, when we set the m up

from sklearn.utils.discovery import all_estimators

estimators = all_estimators(type_filter='classifier')

for name, class_ in estimators:
    try:
        if hasattr(class_(), 'class_weight'):
            print(name)
    except:
        pass

DecisionTreeClassifier
ExtraTreeClassifier
ExtraTreesClassifier
HistGradientBoostingClassifier
LinearSVC
LogisticRegression
LogisticRegressionCV
NuSVC
PassiveAggressiveClassifier
Perceptron
RandomForestClassifier
RidgeClassifier
RidgeClassifierCV
SGDClassifier
SVC


Not all classifiers support class_weight. For those which don't, like GradientBoostingClassifier, we can still use sample_weight when we fit the estimator.

## Demo

In this demo, we are going to introduce the misclassification cost in Logistic Regression, using class_weight and then sample_weight.

In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [4]:
# load data
# only a few observations to speed the computaton

data = pd.read_csv('../kdd2004.csv').sample(10000)

data.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,target
34690,59.59,27.59,-0.54,-26.0,46.5,1573.7,0.94,-1.53,-7.5,-54.5,...,1878.0,0.36,1.94,-2.0,-60.0,500.2,0.72,0.41,0.47,-1
99550,68.07,23.05,1.18,34.0,23.5,4446.1,-1.68,0.63,-4.0,-114.0,...,4277.1,0.04,-3.45,-14.0,-95.0,534.9,1.54,0.05,-0.40,-1
128835,69.31,22.78,1.33,18.0,33.0,1692.2,1.20,-0.19,-7.5,-64.5,...,2957.0,0.55,-0.45,6.0,-38.0,389.3,0.33,0.01,-0.93,-1
25996,51.79,27.27,0.35,8.5,-3.5,977.6,-0.74,0.69,-6.0,-60.0,...,439.1,1.52,0.43,-1.0,-36.0,204.7,1.26,0.29,0.55,-1
46382,79.82,21.98,1.69,40.0,-46.5,1819.4,-0.34,1.81,22.5,-96.0,...,2283.3,-1.66,1.36,4.0,-72.0,893.7,-0.24,0.20,0.53,-1


In [5]:
# imbalanced target

data.target.value_counts() / len(data)

target
-1    0.9909
 1    0.0091
Name: count, dtype: float64

In [6]:
# separate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),  # drop the target
    data['target'],  # just the target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((7000, 74), (3000, 74))

## Using class_weight

In [7]:
# Logistic Regression with class_weight

# we initialize the cost / weights when we set up the transformer

def run_Logit(X_train, X_test, y_train, y_test, class_weight):
    
    # weights introduced here
    logit = LogisticRegression(
        penalty='l2',
        solver='newton-cg',
        random_state=0,
        max_iter=10,
        n_jobs=4,
        class_weight=class_weight # weights / cost
    )
    
    logit.fit(X_train, y_train)

    print('Train set')
    pred = logit.predict_proba(X_train)
    print(
        'Logit roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Test set')
    pred = logit.predict_proba(X_test)
    print(
        'Logit roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [8]:
# evaluate performance of algorithm built
# using imbalanced dataset

run_Logit(X_train,
          X_test,
          y_train,
          y_test,
          class_weight=None)

Train set
Logit roc-auc: 0.9598770408336444
Test set
Logit roc-auc: 0.9263337763432694


In [9]:
# evaluate performance of algorithm built
# cost estimated as imbalance ratio

# 'balanced' indicates that we want same amount of 
# each observation, thus, imbalance ratio

run_Logit(X_train,
          X_test,
          y_train,
          y_test,
          class_weight='balanced')

Train set
Logit roc-auc: 0.9955067716642341
Test set
Logit roc-auc: 0.9592966365322545


In [10]:
# evaluate performance of algorithm built
# cost estimated as imbalance ratio

# alternatively, we can pass a different cost
# in a dictionary, if we know it already

run_Logit(X_train,
          X_test,
          y_train,
          y_test,
          class_weight={-1:1, 1:10})

Train set
Logit roc-auc: 0.9772898917212586
Test set
Logit roc-auc: 0.9435965591272217


Play with the cost and see what you get in terms of performance.

## Using sample_weight

In [11]:
# Logistic Regression + sample_weight

# we pass the weights / cost, when we train the algorithm

def run_Logit(X_train, X_test, y_train, y_test, sample_weight):
    
    logit = LogisticRegression(
        penalty='l2',
        solver='newton-cg',
        random_state=0,
        max_iter=10,
        n_jobs=4,
    )
    
    # costs are passed here
    logit.fit(X_train, y_train, sample_weight=sample_weight)

    print('Train set')
    pred = logit.predict_proba(X_train)
    print(
        'Logit roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Test set')
    pred = logit.predict_proba(X_test)
    print(
        'Logit roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [12]:
# evaluate performance of algorithm built
# using imbalanced dataset

run_Logit(X_train,
          X_test,
          y_train,
          y_test,
          sample_weight=None)

Train set
Logit roc-auc: 0.9598770408336444
Test set
Logit roc-auc: 0.9263337763432694


In [13]:
# evaluate performance of algorithm built
# cost estimated as imbalance ratio

# with numpy.where, we introduce a cost of 99 to
# each observation of the minority class, and 1
# otherwise.

run_Logit(X_train,
          X_test,
          y_train,
          y_test,
          sample_weight=np.where(y_train==1,99,1))

Train set
Logit roc-auc: 0.9955131360103188
Test set
Logit roc-auc: 0.959340450701757


## Conclusion

Cost-sensitive learning has improved the performance of the model.

**HOMEWORK**

Try other machine learning algorithms and other datasets available in imbalanced-learn